In [79]:
import polars as pl


df = pl.read_csv("../data.csv")

In [3]:
df.columns
df.dtypes

[Int64,
 String,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64]

In [4]:
import datetime 

anomalies = [
    (datetime.datetime(2020, 4, 12, 11, 50), datetime.datetime(2020, 4, 12, 23, 30)),
    (datetime.datetime(2020, 4, 18, 0, 0), datetime.datetime(2020, 4, 18, 23, 59)),
    (datetime.datetime(2020, 4, 19, 0, 0), datetime.datetime(2020, 4, 19, 1, 30)),
    (datetime.datetime(2020, 4, 29, 3, 20), datetime.datetime(2020, 4, 29, 4, 0)),
    (datetime.datetime(2020, 4, 29, 22, 0), datetime.datetime(2020, 4, 29, 22, 20)),
    (datetime.datetime(2020, 5, 13, 14, 0), datetime.datetime(2020, 5, 13, 23, 59)),
    (datetime.datetime(2020, 5, 18, 5, 0), datetime.datetime(2020, 5, 18, 5, 30)),
    (datetime.datetime(2020, 5, 19, 10, 10), datetime.datetime(2020, 5, 19, 11, 0)),
    (datetime.datetime(2020, 5, 19, 22, 10), datetime.datetime(2020, 5, 19, 23, 59)),
    (datetime.datetime(2020, 5, 20, 0, 0), datetime.datetime(2020, 5, 20, 20, 0)),
    (datetime.datetime(2020, 5, 23, 9, 50), datetime.datetime(2020, 5, 23, 10, 10)),
    (datetime.datetime(2020, 5, 29, 23, 30), datetime.datetime(2020, 5, 29, 23, 59)),
    (datetime.datetime(2020, 5, 30, 0, 0), datetime.datetime(2020, 5, 30, 6, 0)),
    (datetime.datetime(2020, 6, 1, 15, 0), datetime.datetime(2020, 6, 1, 15, 40)),
    (datetime.datetime(2020, 6, 3, 10, 0), datetime.datetime(2020, 6, 3, 11, 0)),
    (datetime.datetime(2020, 6, 5, 10, 0), datetime.datetime(2020, 6, 5, 23, 59)),
    (datetime.datetime(2020, 6, 6, 0, 0), datetime.datetime(2020, 6, 6, 23, 59)),
    (datetime.datetime(2020, 6, 7, 0, 0), datetime.datetime(2020, 6, 7, 14, 30)),
    (datetime.datetime(2020, 7, 8, 17, 30), datetime.datetime(2020, 7, 8, 19, 0)),
    (datetime.datetime(2020, 7, 15, 14, 30), datetime.datetime(2020, 7, 15, 19, 0)),
    (datetime.datetime(2020, 7, 17, 4, 30), datetime.datetime(2020, 7, 17, 5, 30)),
]

final_anomalies = []
for anomaly in anomalies: 
  new = list(anomaly)
  if anomaly[1].minute == 59:
    new[1] += datetime.timedelta(minutes=1)
  final_anomalies.append(new)

In [5]:
import matplotlib.pyplot as plt 

def plot_ts(data_to_plot: pl.DataFrame, columns: str, time_window: tuple[int],
            title="Series temporales"):
    """
    Funcion que permite plotear series temporales en distintos rangos de tiempo. Las series aparecen superpuestas y en formato HTML que permite hacer zoom.
    
    ...
    
    Attributes
    ---
        :param data_to_plot: Datos que plotear
        :param columns: nombre de las columnas a seleccionar
        :param time_window: instantes que se quieren plotear 
        :param title: titulo del graficp
        
    Returns
    ---
        Gráfico con las series temporales
    """

    #Lectura de los datos
    import pandas as pd

    if data_to_plot.is_empty():
        raise ValueError("DataFrame de entrada vacio")

    if len(columns) < 1:
        raise ValueError("Seleccione alguna columna")

    if time_window[0] > time_window[1]:
        time_window = time_window[::-1]

    df = pd.DataFrame()
    number_of_points = time_window[1] - time_window[0]
    df["instance"] = range(time_window[0], time_window[1])
    df["variable"] = "prediction"
    df["value"] = data_to_plot.select(columns[0])[time_window[0]:time_window[1]]
    if len(columns) > 1:
        index = 0
        for c in columns[0:]:
            values = data_to_plot.select(c).to_series().to_list()[time_window[0]:time_window[1]]
            for i in range(0, number_of_points):
                df.loc[index + i] = [i, c, values[i]]

            index += number_of_points

    #Creacion de la figura con la serie temporal

    plt.figure(1)
    fig = px.line(df, x="instance", y="value", title=title, color="variable")
    fig.update_yaxes(
        fixedrange=False
    )
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(count=2, label="2y", step="year", stepmode="backward"),
                dict(count=3, label="3y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )
    fig.show()

In [81]:
import datetime
df = df.with_columns(pl.col("timestamp").str.to_datetime())
df = df.with_row_index("rownr")

In [7]:
anomalies[1][1] + datetime.timedelta(seconds=59)

datetime.datetime(2020, 4, 18, 23, 59, 59)

In [8]:
def is_anomaly(instance_date, anomalies: list[datetime]):
    flag_anomaly = False
    index = 0
    while not flag_anomaly and index < len(anomalies):
        t = anomalies[index]
        if instance_date >= t[0] and instance_date <= t[1] + datetime.timedelta(seconds=59):
            flag_anomaly = True
        index += 1
    return flag_anomaly


df = df.select(pl.all(), pl.lit(0).alias("is_anomaly"))
for anomaly in final_anomalies:
  df = df.with_columns((pl.col("is_anomaly") + df["timestamp"].is_between(anomaly[0], anomaly[1])))
df = df.select(pl.exclude("is_anomaly"), pl.col("is_anomaly") >= 1)
# df = df.select(pl.all(), pl.col("timestamp").map_elements(lambda x: is_anomaly(x, anomalies), return_dtype=pl.Boolean).alias("is_anomaly"))

In [9]:
# Create transition states
tdf = df.select(pl.all(), pl.col("is_anomaly").shift(-1).alias("next_is_anomaly"), pl.col("Motor_current").gt(0.05).alias("motor_state"))
tdf = tdf.select(pl.all(), (pl.col("is_anomaly") != pl.col("next_is_anomaly")).alias("transition"))
tdf

rownr,,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,is_anomaly,next_is_anomaly,motor_state,transition
u32,i64,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,bool,bool,bool
0,0,2020-02-01 00:00:00,-0.012,9.358,9.34,-0.024,9.358,53.6,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,false,false,false
1,10,2020-02-01 00:00:10,-0.014,9.348,9.332,-0.022,9.348,53.675,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,false,false,false
2,20,2020-02-01 00:00:19,-0.012,9.338,9.322,-0.022,9.338,53.6,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,false,false,false
3,30,2020-02-01 00:00:29,-0.012,9.328,9.312,-0.022,9.328,53.425,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,false,false,false
4,40,2020-02-01 00:00:39,-0.012,9.318,9.302,-0.022,9.318,53.475,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,false,false,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1516943,15169430,2020-09-01 03:59:10,-0.014,8.918,8.906,-0.022,8.918,59.675,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,false,false,false
1516944,15169440,2020-09-01 03:59:20,-0.014,8.904,8.888,-0.02,8.904,59.6,0.045,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,false,false,false
1516945,15169450,2020-09-01 03:59:30,-0.014,8.89,8.876,-0.022,8.892,59.6,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,false,false,false


In [10]:
# Create groups based on transition, there should be 42 transitions
tdf = tdf.select(pl.all(), pl.col("transition").cum_sum().alias("groups"))
tdf["groups"].max()

32

In [29]:
gdf = tdf.select(pl.all(), pl.col("groups").shift(-1).alias("next_group"), pl.col("motor_state").shift(-1).alias("next_motor_state"))
# Filter a ON switch
fdf = gdf.filter(pl.col("motor_state") != pl.col("next_motor_state")).with_columns(pl.col("timestamp").shift(-1).alias("next_timestamp"))
# Filter out different time groups
fdf = fdf.filter(pl.col("groups") == pl.col("next_group"))
# Calculate the duration
fdf = fdf.with_columns((pl.col("next_timestamp") - pl.col("timestamp")).alias("duration")).filter(pl.col("duration").is_not_null())
print(fdf["duration"].dt.total_seconds().max(), fdf["duration"].dt.total_seconds().min(), fdf["duration"].dt.total_seconds().mean(), fdf["duration"].dt.total_seconds().mode()[0])

274375 9 885.96540941018 545


In [12]:
fdf.filter(pl.col("duration").eq(pl.col("duration").max()))

rownr,,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,is_anomaly,next_is_anomaly,motor_state,transition,groups,next_group,next_motor_state,next_timestamp,duration
u32,i64,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,bool,bool,bool,u32,u32,bool,datetime[μs],duration[μs]
890739,8907390,2020-06-05 09:48:20,-0.014,8.154,8.142,-0.018,8.156,60.575,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,false,false,false,24,24,true,2020-06-08 14:01:15,3d 4h 12m 55s


In [13]:
import plotly.express as px
data_anomaly = df.filter(pl.col("timestamp").is_between(
datetime.datetime(2020,6,5,00,00,00), datetime.datetime(2020,6,8,15,00,00)
))
px.line(data_anomaly, x="timestamp", y=["TP3", "Motor_current"])

In [14]:
data_anomaly.select(pl.col("is_anomaly"), pl.col("is_anomaly").shift(-1).alias("next_is_anomaly"), pl.col("timestamp"), pl.col("timestamp").shift(-1).alias("next_timestamp")).filter(~pl.col("is_anomaly").eq(pl.col("next_is_anomaly")))

is_anomaly,next_is_anomaly,timestamp,next_timestamp
bool,bool,datetime[μs],datetime[μs]
false,true,2020-06-05 09:59:54,2020-06-05 10:00:04
true,false,2020-06-07 14:19:39,2020-06-08 11:48:04


In [35]:
gdf = tdf.select(pl.all(), pl.col("groups").shift(-1).alias("next_group"), pl.col("motor_state").shift(-1).alias("next_motor_state"))
# Filter a ON switch
durations = []
for label, group in gdf.filter(pl.col("groups") == pl.col("next_group")).group_by("groups"):
  tmp = group.filter(pl.col("motor_state") != pl.col("next_motor_state")).with_columns(pl.col("timestamp").shift(-1).alias("next_timestamp"))
  tmp = tmp.with_columns((pl.col("next_timestamp") - pl.col("timestamp")).alias("duration")).filter(pl.col("duration").is_not_null())
  durations += tmp["duration"].to_list()
durations = [x.total_seconds() for x in durations]
print(max(durations), min(durations), sum(durations) / len(durations), durations[len(durations)//2])

223982.0 9.0 851.5546991666265 525.0


In [34]:
subdf = df.filter(pl.col("timestamp").lt(anomalies[0][0]))
subdf = subdf.with_columns(pl.col("Motor_current").lt(0.05).alias("motor_state"))
subdf.with_columns(pl.col("motor_state").shift(-1).alias("next_motor_state")).filter(pl.col("motor_state") != pl.col("next_motor_state"))["timestamp"].diff().dt.total_seconds().mode()[0]

535

In [30]:
durations = []
subdf = df.filter(pl.col("timestamp").lt(final_anomalies[0][0]))
subdf = subdf.with_columns(pl.col("Motor_current").lt(0.05).alias("motor_state"))
subdf = subdf.with_columns(pl.col("motor_state").shift(-1).alias("next_motor_state")).filter(pl.col("motor_state") != pl.col("next_motor_state"))
subdf = subdf.with_columns(pl.col("timestamp").diff().dt.total_seconds().alias("duration")).filter(pl.col("duration").is_not_null())
print(len(durations), len(subdf))
durations += subdf["duration"].to_list()
for anomaly, next_anomaly, next_next_anomaly in zip(final_anomalies, final_anomalies[1:], final_anomalies[2:]):
  subdf = df.filter(
    (pl.col("timestamp").lt(next_anomaly[0]) & (pl.col("timestamp").gt(anomaly[1])))
  )
  subdf = subdf.with_columns(pl.col("Motor_current").lt(0.05).alias("motor_state"))
  subdf = subdf.with_columns(pl.col("motor_state").shift(-1).alias("next_motor_state")).filter(pl.col("motor_state") != pl.col("next_motor_state"))
  subdf = subdf.with_columns(pl.col("timestamp").diff().dt.total_seconds().alias("duration")).filter(pl.col("duration").is_not_null())
  print(len(durations), len(subdf))
  durations += subdf["duration"].to_list()
  subdf = df.filter(
    (pl.col("timestamp").gt(next_anomaly[1]) & (pl.col("timestamp").lt(next_next_anomaly[0])))
  )
  subdf = subdf.with_columns(pl.col("Motor_current").lt(0.05).alias("motor_state"))
  subdf = subdf.with_columns(pl.col("motor_state").shift(-1).alias("next_motor_state")).filter(pl.col("motor_state") != pl.col("next_motor_state"))
  subdf = subdf.with_columns(pl.col("timestamp").diff().dt.total_seconds().alias("duration")).filter(pl.col("duration").is_not_null())
  print(len(durations), len(subdf))
  durations += subdf["duration"].to_list()
print(max(durations), min(durations), sum(durations) / len(durations), sorted(durations)[len(durations)//2])

0 6331
6331 294
6625 0
6625 0
6625 622
7247 622
7869 77
7946 77
8023 1032
9055 1032
10087 365
10452 365
10817 202
11019 202
11221 88
11309 88
11397 0
11397 0
11397 387
11784 387
12171 539
12710 539
13249 0
13249 0
13249 291
13540 291
13831 166
13997 166
14163 231
14394 231
14625 0
14625 0
14625 0
14625 0
14625 3491
18116 3491
21607 759
22366 759
23125 195
223982 9 890.077873070326 555


In [83]:
df = df.with_columns(pl.col("timestamp").diff().cum_sum().alias("diff"))

In [84]:
df = df.with_columns((pl.col("diff").dt.total_seconds() // 1400).alias("id"))
df = df.with_columns(pl.coalesce(pl.col("id").cast(pl.UInt32), 0).alias("id"))

In [85]:
df.group_by("id").len()

id,len
u32,u32
5380,141
8926,116
11549,141
3832,141
6467,141
…,…
4954,141
10328,142
13073,141


In [86]:
px.line(df[:10000], x="timestamp", y="TP3", color="id")

In [76]:
with pl.Config(tbl_rows=1000):
  print(df.filter(pl.col("timestamp").gt(datetime.datetime(2020,2, 1, 19, 40, 00)) & pl.col("timestamp").lt(datetime.datetime(2020, 2, 2, 6, 00, 00))))

shape: (549, 21)
┌───────┬───────┬─────────────────┬────────┬───┬────────────────┬────────────┬───────────────┬─────┐
│ rownr ┆       ┆ timestamp       ┆ TP2    ┆ … ┆ Caudal_impulse ┆ is_anomaly ┆ diff          ┆ id  │
│ ---   ┆ ---   ┆ ---             ┆ ---    ┆   ┆ s              ┆ ---        ┆ ---           ┆ --- │
│ u32   ┆ i64   ┆ datetime[μs]    ┆ f64    ┆   ┆ ---            ┆ bool       ┆ duration[μs]  ┆ u32 │
│       ┆       ┆                 ┆        ┆   ┆ f64            ┆            ┆               ┆     │
╞═══════╪═══════╪═════════════════╪════════╪═══╪════════════════╪════════════╪═══════════════╪═════╡
│ 7113  ┆ 71130 ┆ 2020-02-01      ┆ -0.012 ┆ … ┆ 1.0            ┆ false      ┆ 19h 40m 4s    ┆ 50  │
│       ┆       ┆ 19:40:04        ┆        ┆   ┆                ┆            ┆               ┆     │
│ 7114  ┆ 71140 ┆ 2020-02-01      ┆ -0.018 ┆ … ┆ 1.0            ┆ false      ┆ 23h 15m 33s   ┆ 59  │
│       ┆       ┆ 23:15:33        ┆        ┆   ┆                ┆         

In [78]:
df.filter(pl.col("timestamp").diff().dt.total_seconds().gt(10))

rownr,,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,is_anomaly,diff,id
u32,i64,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,duration[μs],u32
4654,46540,2020-02-01 12:53:47,-0.012,9.124,9.108,-0.012,9.13,59.675,0.0375,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,12h 53m 47s,33
7114,71140,2020-02-01 23:15:33,-0.018,9.99,9.98,-0.022,9.992,44.85,3.97,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,23h 15m 33s,59
7144,71440,2020-02-02 04:34:27,9.952,9.63,-0.012,-0.024,9.63,31.975,6.275,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,false,1d 4h 34m 27s,73
8254,82540,2020-02-02 07:41:52,-0.012,8.31,8.294,-0.022,8.312,53.1,3.8125,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,1d 7h 41m 52s,81
31804,318040,2020-02-05 00:43:00,10.412,10.032,-0.012,-0.022,10.03,56.6,6.3325,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,false,4d 43m,248
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1502468,15024680,2020-08-30 04:26:10,9.264,8.844,-0.01,-0.022,8.842,51.775,6.0075,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,false,211d 4h 26m 10s,13033
1509758,15097580,2020-08-31 01:49:32,-0.012,9.878,9.866,-0.02,9.878,64.45,3.7775,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,212d 1h 49m 32s,13088
1511498,15114980,2020-08-31 06:42:31,-0.012,9.596,9.58,-0.022,9.596,65.775,3.6775,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,212d 6h 42m 31s,13100
